# Week 2. Take-Home Project
## 寫一隻ptt爬蟲將91App資料與當天地方版的貼文數量連結

### 0. 引入module
使用到的爬蟲函式： <br>
`ptt_region_crawler.get_post_num(region, date, verbose=True, init_index=None, return_index=False)`

#### Parameters
* **region: string**<br>
    the name of the ptt site
* **date: datetime-like string, timestamp**<br> 
the date which the post number is to be fetched
* **verbose: boolean**<br>
if True then show the details when crawling
* **init_index: None, int**<br>
the page index to start crawling
* **return_index: boolean**<br>
if True then return the page index when the crawler ends

#### Returns
If `return_index` is False then return int for the fetched `post_num`, or return tuple `(post_num, page_index)` when True.


In [1]:
import pandas as pd
import ptt_region_crawler as ptt_crawl

### 1. 讀入資料
#### 因為要做完整個91App資料集的ptt貼文數欄位費時極久，這邊先取台北市的資料作為範例

In [2]:
data = pd.read_csv('../91APPdataset/Orders.csv')
taipei_data = data[data.City == '台北市'].dropna(subset=['District'])
taipei_data.info()

/home/kevin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 541164 entries, 35 to 5495274
Data columns (total 28 columns):
DateId                         541164 non-null int64
MemberId                       541164 non-null int64
OrderGroupCode                 541164 non-null object
TrackSourceTypeDef             541164 non-null object
TrackDeviceTypeDef             541164 non-null object
PayProfileTypeDef              541164 non-null object
SalesOrderSlaveId              541164 non-null int64
SalePageId                     541164 non-null int64
IsMajor                        541164 non-null bool
IsGift                         541164 non-null bool
IsSalePageGift                 541164 non-null bool
Quantity                       541164 non-null int64
UnitPrice                      541164 non-null float64
PromotionDiscount              541164 non-null float64
ECouponId                      541164 non-null int64
ECouponDiscount                541164 non-null float64
SalesOrderSlaveTotalPayment    5

### 2. 欄位提取
#### 取出資料中的行政區和時間欄位，並製作ptt版銘與區名連結的dictionary

In [3]:
taipei_district_date = taipei_data[['District', 'DateId']].drop_duplicates()
taipei_district_date['new_DateId'] = pd.to_datetime(taipei_district_date['DateId'], format="%Y%m%d")
taipei_district_list = taipei_district_date['District'].unique()
taipei_district_group = taipei_district_date.groupby('District')

In [4]:
district_hash ={ 
    "北投區": "BigPeitou", 
    "士林區": "BigShiLin",
    "大安區": "Daan",
    "大同區": "Datong",
    "信義區": "HsinYi", 
    "南港區": "Nangang",
    "內湖區": "Neihu",
    "松山區": "SongShan",
    "萬華區": "Wanhua",
    "文山區": "Wen-Shan",
    "中山區": "Zhongshan",
    "中正區": "Zhongzheng"
}

### 3. 開始爬蟲
#### 因為module寫的時候沒有把bug全部抓完，只好在迴圈內設置例外處理，正確抓取的貼文數才不至於在例外被拋出的時候同時被丟掉

In [5]:
group_list = []
for i in taipei_district_list:
    post_count = []
    region = district_hash[i]
    temp_index = None
    temp_data = taipei_district_group.get_group(i).sort_values('new_DateId', ascending=False)
    date = temp_data['new_DateId']
    for t in list(date):
        try:
            (num, temp_index) = ptt_crawl.get_post_num(region, t, verbose=False, init_index=temp_index, return_index=True)
        except Exception as e:
            print(i , t, repr(e))
            (num, temp_index) = (-1, temp_index)
        finally:
            post_count.append(num)
    temp_data['post_count'] = post_count
    group_list = group_list + [temp_data]

中山區 2017-06-21 00:00:00 SSLError(SSLError(SSLError("bad handshake: SysCallError(104, 'ECONNRESET')",),),)
中山區 2017-06-18 00:00:00 SSLError(SSLError(SSLError("bad handshake: SysCallError(104, 'ECONNRESET')",),),)
中山區 2016-12-17 00:00:00 TypeError("'NoneType' object is not subscriptable",)
中山區 2016-10-01 00:00:00 TypeError("'NoneType' object is not iterable",)
中山區 2016-06-23 00:00:00 TypeError("'NoneType' object is not subscriptable",)
中山區 2016-04-17 00:00:00 TypeError("'NoneType' object is not iterable",)
中山區 2016-02-04 00:00:00 IndexError('list index out of range',)
大安區 2017-09-10 00:00:00 TypeError("'NoneType' object is not iterable",)
大安區 2017-06-08 00:00:00 TypeError("'NoneType' object is not subscriptable",)
萬華區 2017-05-23 00:00:00 IndexError('list index out of range',)
萬華區 2017-05-18 00:00:00 IndexError('list index out of range',)
北投區 2017-06-19 00:00:00 IndexError('list index out of range',)


#### 把未正確抓取貼文數的時間地點取出， 重新執行爬蟲

In [10]:
post_count_data = pd.concat(group_list, axis=0)
post_count_data.to_csv("post_count_data.csv")
error_data = post_count_data[post_count_data['post_count']==-1]
error_data

,District,DateId,new_DateId,post_count
1341028,中山區,20170621,2017-06-21,-1
1253967,中山區,20170618,2017-06-18,-1
5002219,中山區,20161217,2016-12-17,-1
4069827,中山區,20161001,2016-10-01,-1
2658606,中山區,20160623,2016-06-23,-1
1923732,中山區,20160417,2016-04-17,-1
1101564,中山區,20160204,2016-02-04,-1
2240765,大安區,20170910,2017-09-10,-1
1197583,大安區,20170608,2017-06-08,-1
891873,萬華區,20170523,2017-05-23,-1


In [11]:
error_group_list = []
for i in error_data['District'].unique():
    post_count = []
    region = district_hash[i]
    temp_index = None
    temp_data = error_data.groupby('District').get_group(i).sort_values('new_DateId', ascending=False)
    date = temp_data['new_DateId']
    for t in list(date):
        try:
            (num, temp_index) = ptt_crawl.get_post_num(region, t, verbose=False, init_index=temp_index, return_index=True)
        except Exception as e:
            print(i , t, temp_index, repr(e))
            (num, temp_index) = (-1, temp_index)
        finally:
            post_count.append(num)
    temp_data['post_count'] = post_count
    error_group_list = error_group_list + [temp_data]

In [12]:
error_data = pd.concat(error_group_list, axis=0)
error_data

,District,DateId,new_DateId,post_count
1341028,中山區,20170621,2017-06-21,3
1253967,中山區,20170618,2017-06-18,0
5002219,中山區,20161217,2016-12-17,3
4069827,中山區,20161001,2016-10-01,1
2658606,中山區,20160623,2016-06-23,3
1923732,中山區,20160417,2016-04-17,0
1101564,中山區,20160204,2016-02-04,7
2240765,大安區,20170910,2017-09-10,2
1197583,大安區,20170608,2017-06-08,1
891873,萬華區,20170523,2017-05-23,4


### 4.資料合併
#### 將新抓取的正確資料合併回原本抓取的貼文數資料，再併入台北市用戶消費資料

In [13]:
post_count_data = pd.concat([post_count_data[post_count_data.post_count != -1], error_data], axis=0)

In [18]:
taipei_result = pd.merge(taipei_data, post_count_data, on=['District', 'DateId'])
taipei_result.to_csv('taipei_result.csv')
taipei_result.sample(10)

,DateId,MemberId,OrderGroupCode,TrackSourceTypeDef,TrackDeviceTypeDef,PayProfileTypeDef,SalesOrderSlaveId,SalePageId,IsMajor,IsGift,...,District,ZipCode,StoreName,StoreId,DeliverTypeDef,StatusDef,ReturnGoodsDateId,CauseDef,new_DateId,post_count
293515,20160629,1260916,b'\x97\xf3\x88A\x13=-\x86\x07W\x1c\xba.\xaf6 %...,Web,Mobile,Family,16134748,2275516,True,False,...,大安區,106.0,全家通和店,4420,Family,Finish,NaN,NaN,2016-06-29,4
332175,20160711,1162212,b'\x81\xd5IQ\\\xd40\xf08\xa7\xf4\xebo\xcdMX\xe...,iOSApp,Mobile,Family,16758757,1830694,True,False,...,松山區,105.0,全家新東光店,14918,Family,Finish,NaN,NaN,2016-07-11,1
286461,20160605,1699614,b')\xf5\n^\x01{\xdb\x8e\x88\x1b4\xa8H\xe8[\xd2...,iOSApp,Mobile,Family,15082072,1942470,True,False,...,松山區,105.0,全家興北店,1539,Family,Finish,NaN,NaN,2016-06-05,1
123042,20170515,4055339,"b""\xab\x9e\x06'L\x18\x04`\xbd\x19\xd1\xfe\xcf\...",Web,Mobile,Family,35295722,3189623,True,False,...,中山區,104.0,全家聚葉店,6420,Family,Finish,NaN,NaN,2017-05-15,3
257656,20160502,3215088,"b""\x82u\x0f\xef\xd7p\xf6K\xd3\xbf\x12\xc4\xed\...",iOSApp,Mobile,SevenEleven,13510982,2012280,True,False,...,中山區,104.0,新錦祥門市,975375,SevenEleven,Finish,NaN,NaN,2016-05-02,1
379940,20160816,2881666,"b""2L\xdc'2\x81\xbds\r\x86\xd5\x05\xefw\xa52\xd...",Web,Mobile,SevenEleven,18500339,2294397,True,False,...,中山區,104.0,鑫長安門市,970749,SevenEleven,Finish,NaN,NaN,2016-08-16,6
300476,20160620,3335508,b'5b\x8bM\xcb\x88\x8b\n\xe8\x9e\xea\xdb\xcfP\x...,iOSApp,Mobile,Family,15757494,2229763,True,False,...,中正區,100.0,全家南陽店,000112,Family,Finish,NaN,NaN,2016-06-20,0
410576,20161003,2446996,b'\xd39\xfe\x94\xdd\xbc\xfd|)pU\xda\xeb\x81\x0...,Web,Mobile,SevenEleven,20973262,2551547,True,False,...,士林區,111.0,洲美門市,170912,SevenEleven,Overdue,NaN,NaN,2016-10-03,0
348156,20160803,2931957,b'\xad\xd8\xd6Vh8e\xda\xa1f\xda\xc4Y\xf0w\x8a\...,Web,Mobile,Family,17835917,2177315,True,False,...,內湖區,114.0,全家康福店,4453,Family,Finish,NaN,NaN,2016-08-03,8
95919,20170504,619660,"b""\x1cc\xee\x8c\xce\xfa\xa2\x16\xf2\x8e\xca\x1...",iOSApp,Mobile,CreditCardOnce,34465203,3048879,True,False,...,大安區,106.0,NaN,NaN,Home,Finish,NaN,NaN,2017-05-04,3
